In [3]:
import langgraph
import boto3
import json
import pandas as pd
import os
import ast
from typing import TypedDict, Dict, List
import requests
from langgraph.graph import StateGraph, END, START, StateGraph
import logging
from typing import TypedDict, Dict, List
import requests, json

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
!pip install langgraph
!pip install boto3
!pip install grandalf

In [4]:
# Load the API key from the secret.json file
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    aws_access_key_id = secrets.get('AWS_ACCESS_KEY')
    aws_secret_access_key = secrets.get('AWS_SECRET_ACCESS_KEY')
    wiki_key = secrets.get('WIKIPEDIA_KEY')

# Create a session using your AWS credentials
session = boto3.Session(aws_access_key_id=aws_access_key_id,
                        aws_secret_access_key=aws_secret_access_key,
                        region_name='us-west-2'
)

# Create a Bedrock client
bedrock_client = session.client('bedrock-runtime')
model_id = 'anthropic.claude-3-opus-20240229-v1:0'


def call_claude(messages: list, model_id: str) -> str:
    body = {
        "messages": messages,
        "max_tokens": 300,
        "anthropic_version": "bedrock-2023-05-31"
    }

    response = bedrock_client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(body)
    )

    response_body = json.loads(response['body'].read())
    return response_body['content'][0]['text']

def get_wikipedia_intro(title: str) -> Dict[str, List]:
    """
    Fetch the introductory paragraph(s) from a Wikipedia page for a given title.

    Args:
        title (str): The Wikipedia page title to query.

    Returns:
        Dict[str, Any]: A dictionary containing:
            - 'information': List[str] of extracted intro text lines.
            - 'metadata': str indicating status ('one entry', 'multiple entries',
                          'no info', or 'respond error').
    """

    headers = {'Authorization': wiki_key, 'User-Agent': 'thesis'}
    endpoint = 'https://en.wikipedia.org/w/api.php'
    params = {
        'format': 'json',
        'action': 'query',
        'prop': 'extracts|pageprops',
        'exintro': 1,
        'explaintext': 1,
        'redirects': 1,
        'titles': title,
    }

    response = requests.get(endpoint, params=params, headers=headers, timeout=10)

    if not response.ok:
        return {'information': [], 'metadata': 'respond error'}

    data = response.json()
    page = next(iter(data['query']['pages'].values()))
    extract = page.get('extract', '')
    pageprops = page.get('pageprops', [])
    
    is_disambiguation = any('disambiguation' in str(cat).lower() for cat in pageprops)

    if not extract:
        return {'information': [], 'metadata': 'no info'}

    # Happy path return — only set what's needed
    if is_disambiguation:
        metadata = 'multiple entries'
        intro_list = extract
    else:
        intro_list = get_wikipedia_summary(title)
        metadata = 'one entry'

    return {'introduction':intro_list, 'information': extract, 'metadata': metadata}


def get_wikipedia_summary(title: str) -> str:
    """
    Fetch a short summary for a Wikipedia page.

    Args:
        title (str): The Wikipedia page title.

    Returns:
        str: The short summary text, or an empty string if not found.
    """
    # Replace spaces with underscores for the URL
    title = title.replace(" ", "_")
    url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"

    response = requests.get(url, headers={"User-Agent": "thesis"}, timeout=10)

    if not response.ok:
        return ""

    data = response.json()
    return data.get("extract", "")


In [7]:
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

# State
class AgentState(TypedDict):
    text: str
    name: str
    contains_name: bool
    wiki_search: Dict[str, List]
    name_mapping: bool
    tailored_summary: str

# Nodes
def detect_name_node(state: AgentState) -> AgentState:
    sentence = state["text"]
    entity = state["name"]
    messages = [
        {"role": "user",
         "content": f"""Does entity '{entity}' in the following text 
                        refer to a person's name? Respond only with true or false.
                        '{sentence}'"""}
    ]
    result = call_claude(messages, model_id).strip().lower()
    state["contains_name"] = result == "true"
    if state["contains_name"]:
        logger.info("Name detected")
    else:
        logger.info("No name detected")
    
    return state


def get_wikipedia_intro_node(state: AgentState) -> AgentState:
    state["wiki_search"] = get_wikipedia_intro(state["name"])
    logging.info("Information retrieved")
    return state


# Conditional routing
def route_after_detect(state: AgentState) -> str:
    return "has_name" if state["contains_name"] else "no_name"


def check_info_relevancy_node(state: AgentState) -> AgentState:
    sentence = state["text"]
    wiki_info = " ".join(state["wiki_search"]['introduction'])
    messages = [
        {"role": "user",
         "content": f"""Is the following Wikipedia info relevant to the entity '{state['name']}' 
                        in the context of this text? Respond only with true or false.
                        TEXT: {sentence}
                        WIKI INFO: {wiki_info}"""}
    ]
    result = call_claude(messages, model_id).strip().lower()
    state["name_mapping"] = result == "true"
    if state["name_mapping"]:
        logger.info("Name matched with retrieved info")
    else:
        logger.info("Name didn't match with retrieved info")

    return state


def tailor_summary_node(state: AgentState) -> AgentState:
    sentence = state["text"]
    wiki_info = " ".join(state["wiki_search"]['information'])
    messages = [
        {"role": "user",
         "content": f"""You are given:
            1. A sentence from a text.
            2. Wikipedia information about the entity '{state['name']}'.
            Your task:
            - Assume the reader (another LLM) does not know who this entity is.
            - From the Wikipedia info, select only the facts needed to:
              a) Identify who the entity is.
              b) Mention relevant information that could be used as a context 
              for the sentence.
            - Ignore unrelated details that do not help interpret the sentence.
            - Combine these into a single, concise, self-contained summary that
            would let the reader fully understand the reference in the sentence.

            TEXT: {sentence}
            WIKIPEDIA INFO: {wiki_info}

            Return only the summary, no extra commentary.
            """}
    ]
    result = call_claude(messages, model_id).strip()
    state["tailored_summary"] = result
    logger.info("Tailored information to text.")
    
    return state

# Graph build
myGraph = StateGraph(AgentState)

myGraph.add_node("detect_name_node", detect_name_node)
myGraph.add_node("get_wikipedia_intro_node", get_wikipedia_intro_node)
myGraph.add_node("check_info_relevancy_node", check_info_relevancy_node)
myGraph.add_node("tailor_summary_node", tailor_summary_node)

myGraph.add_edge(START, "detect_name_node")

# Conditional edges with explicit mapping
myGraph.add_conditional_edges(
    "detect_name_node",
    route_after_detect,
    {
        "has_name": "get_wikipedia_intro_node",
        "no_name": END
    }
)

myGraph.add_edge("get_wikipedia_intro_node", "check_info_relevancy_node")
myGraph.add_edge("check_info_relevancy_node", "tailor_summary_node")
myGraph.add_edge("tailor_summary_node", END)

compiled_graph = myGraph.compile()
print(compiled_graph.get_graph().draw_ascii())

result = compiled_graph.invoke({"name": "John",
                     "text": "I wish I was an Asian guy"})
print(result)

                          +-----------+              
                          | __start__ |              
                          +-----------+              
                                 *                   
                                 *                   
                                 *                   
                       +------------------+          
                       | detect_name_node |          
                       +------------------+          
                       ...               ....        
                    ...                      ...     
                  ..                            ...  
+--------------------------+                       ..
| get_wikipedia_intro_node |                        .
+--------------------------+                        .
              *                                     .
              *                                     .
              *                                     .
+---------------------------

2025-09-11 19:04:54,449 - INFO - No name detected


{'text': 'I wish I was an Asian guy', 'name': 'John', 'contains_name': False}


In [83]:
image_data = compiled_graph.get_graph().draw_mermaid_png()
with open("information_extraction_graph.png", mode="wb") as f:
    f.write(image_data)